In [9]:
import os
import cv2 as cv
import pandas as pd
import numpy as np

BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, "data")

if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

# ===============================
# Load Dataset
# ===============================
file_path = os.path.join(DATA_DIR, "stock_data.csv")
df = pd.read_csv(file_path)

df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')

# Create Trend Target
# 1 = Price Up, 0 = Price Down
df['Target'] = np.where(df['Close'].shift(-1) > df['Close'], 1, 0)

# Remove last row (NaN target)
df = df.dropna()

# ===============================
# Feature Selection
# ===============================
features = df[['Open', 'High', 'Low', 'Close', 'Volume']].values
target = df['Target'].values

split_index = int(len(features) * 0.8)

X_train = features[:split_index]
X_test = features[split_index:]

y_train = target[:split_index]
y_test = target[split_index:]

# ===============================
# Normalization using OpenCV
# ===============================
X_train = cv.normalize(X_train, None, 0, 1, cv.NORM_MINMAX)
X_test = cv.normalize(X_test, None, 0, 1, cv.NORM_MINMAX)

# ===============================
# Output Check
# ===============================
print("Train Shape:", X_train.shape)
print("Test Shape:", X_test.shape)
print("Train Target Distribution:", np.bincount(y_train))
print("Test Target Distribution:", np.bincount(y_test))

Train Shape: (2364, 5)
Test Shape: (592, 5)
Train Target Distribution: [1163 1201]
Test Target Distribution: [262 330]


In [4]:
pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/40.2 MB ? eta -:--:--
   - -------------------------------------- 1.0/40.2 MB 6.1 MB/s eta 0:00:07
   - -------------------------------------- 1.8/40.2 MB 4.4 MB/s eta 0:00:09
   -- ------------------------------------- 2.6/40.2 MB 4.4 MB/s eta 0:00:09
   --- ------------------------------------ 3.4/40.2 MB 4.3 MB/s eta 0:00:09
   --- ------------------------------------ 3.9/40.2 MB 4.2 MB/s eta 0:00:09
   --- ------------------------------------ 3.9/40.2 MB 4.2 MB/s eta 0:00:09
   ---- ----------------------------------- 4.2/40.2 MB 2.8 MB/s eta 0:00:13
   ------ --------------------------------- 6.3/40.2 MB 3.9 MB/s eta 0:00:09
   ------- -------------------------------- 7.1/40.2 MB 3.9 MB/s eta 0:00:09
   ------- -------------------------------- 7.9/40.2 MB 3.8 MB/s eta 0:00:09
   -------- ------------------------------- 8.1/40.2 MB 3.8 MB/s eta 0:00:09
   --

In [10]:

window_short = 5    # Short-term moving average
window_long = 20    # Long-term moving average

# Calculate moving averages
df['MA_Short'] = df['Close'].rolling(window=window_short).mean()
df['MA_Long'] = df['Close'].rolling(window=window_long).mean()

# Drop NaN values created by rolling
df.dropna(inplace=True)

# Predict Trend: if short MA > long MA → Up (1), else Down (0)
df['Predicted'] = np.where(df['MA_Short'] > df['MA_Long'], 1, 0)

# ===============================
# Evaluate Accuracy
# ===============================
# Align with Target column
actual = df['Target'].values[-len(df['Predicted']):]
predicted = df['Predicted'].values

accuracy = (predicted == actual).mean() * 100
print(f"Directional Accuracy: {accuracy:.2f}%")

# ===============================
# Optional: Show Sample Predictions
# ===============================
result = df[['Date', 'Close', 'Target', 'Predicted']].tail(10)
print(result)


Directional Accuracy: 51.52%
           Date        Close  Target  Predicted
2946 2022-03-11   795.349976       0          0
2947 2022-03-14   766.369995       1          0
2948 2022-03-15   801.890015       1          0
2949 2022-03-16   840.229980       1          0
2950 2022-03-17   871.599976       1          0
2951 2022-03-18   905.390015       1          1
2952 2022-03-21   921.159973       1          1
2953 2022-03-22   993.979980       1          1
2954 2022-03-23   999.109985       1          1
2955 2022-03-24  1013.919983       0          1
